In [1]:
base_dir='dataset/annotations/annotations/tcia-lidc-xml/'

In [2]:
import os

c = 0
# Verify XML file paths
for root_dir, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".xml"):
            c += 1
            # print(os.path.join(root_dir, file))  # Print XML file paths
print(c)


1319


In [4]:
import xml.etree.ElementTree as ET

file_path='dataset/annotations/annotations/tcia-lidc-xml/157/158.xml'

# Parse the XML file
tree = ET.parse(file_path)
root = tree.getroot()

# Print the full XML structure
print(ET.tostring(root, encoding="utf-8").decode("utf-8"))

<ns0:LidcReadMessage xmlns:ns0="http://www.nih.gov" uid="1.3.6.1.4.1.14519.5.2.1.6279.6001.1308168927827.0">
 <ns0:ResponseHeader>
  <ns0:Version>1.7</ns0:Version>
  <ns0:MessageId>1151420</ns0:MessageId>
  <ns0:DateRequest>2006-06-05</ns0:DateRequest>
  <ns0:TimeRequest>17:05:26</ns0:TimeRequest>
  <ns0:RequestingSite>removed</ns0:RequestingSite>
  <ns0:ServicingSite>removed</ns0:ServicingSite>
  <ns0:TaskDescription>Second unblinded read</ns0:TaskDescription>
  <ns0:CtImageFile>removed</ns0:CtImageFile>
  <ns0:SeriesInstanceUid>1.3.6.1.4.1.14519.5.2.1.6279.6001.303494235102183795724852353824</ns0:SeriesInstanceUid>
  <ns0:StudyInstanceUID>1.3.6.1.4.1.14519.5.2.1.6279.6001.339170810277323131167631068432</ns0:StudyInstanceUID>
  <ns0:DateService>2006-06-05</ns0:DateService>
  <ns0:TimeService>17:05:26</ns0:TimeService>
  <ns0:ResponseDescription>1 - Reading complete</ns0:ResponseDescription>
  <ns0:ResponseComments>Merged, reader anonymized, unblinded responses</ns0:ResponseComments>
 

In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd


# Function to parse an XML file
def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    nodule_data = []
    ns = {"nih": "http://www.nih.gov"}

    for reading_session in root.findall(".//nih:readingSession", ns):
        for unblinded_nodule in reading_session.findall(
            ".//nih:unblindedReadNodule", ns
        ):
            # Extract Nodule ID
            nodule_id = unblinded_nodule.find("nih:noduleID", ns)
            nodule_id = (
                nodule_id.text.strip()
                if nodule_id is not None and nodule_id.text
                else "Missing"
            )

            # Extract Malignancy
            malignancy = unblinded_nodule.find(".//malignancy")
            if malignancy is None:
                malignancy = unblinded_nodule.find(".//nih:malignancy", ns)
            malignancy = (
                malignancy.text.strip()
                if malignancy is not None and malignancy.text
                else "Missing"
            )

            # Extract bounding box
            x_coords = []
            y_coords = []

            for edge in unblinded_nodule.findall(".//edgeMap"):
                x = edge.find("xCoord")
                y = edge.find("yCoord")
                if x is not None and y is not None:
                    x_coords.append(int(x.text))
                    y_coords.append(int(y.text))

            if not x_coords or not y_coords:
                for edge in unblinded_nodule.findall(".//nih:edgeMap", ns):
                    x = edge.find("nih:xCoord", ns)
                    y = edge.find("nih:yCoord", ns)
                    if x is not None and y is not None:
                        x_coords.append(int(x.text))
                        y_coords.append(int(y.text))

            if x_coords and y_coords and len(x_coords) > 1:
                bbox = [min(x_coords), min(y_coords), max(x_coords), max(y_coords)]
            else:
                bbox = "Invalid"

            # Debugging outputs
            if malignancy == "Missing":
                print("Malignancy Missing in:", file_path)
                print(ET.tostring(unblinded_nodule, encoding="utf8").decode("utf8"))
            if bbox == "Invalid":
                print("EdgeMap missing in:", file_path)

            # Append to list
            nodule_data.append([file_path, nodule_id, malignancy, bbox])

    return nodule_data


# Directory containing XML files
base_dir = "dataset/"

# Collect data from all XML files
all_nodule_data = []
xml_count = 0

for root_dir, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".xml"):
            file_path = os.path.join(root_dir, file)
            xml_count += 1
            print(f"Processing: {file_path}")
            all_nodule_data.extend(parse_xml(file_path))

print(f"Total XML files processed: {xml_count}")

# Convert to DataFrame
df = pd.DataFrame(
    all_nodule_data, columns=["File", "NoduleID", "Malignancy", "BoundingBox"]
)
print(df.head())
print(df.shape)

Total XML files processed: 0
Empty DataFrame
Columns: [File, NoduleID, Malignancy, BoundingBox]
Index: []
(0, 4)
